In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="2944a580-6c5f-4258-8c86-b9c5de957998"
RESOURCE_GROUP="calebebraga08-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

In [ ]:
TRAIN_DATAS = [
    "vrex_2009_2010_2011_2012_2013_.csv",
    "vrex_2014_2015_2016_2017_2018_.csv",
]

TEST_DATAS = [
    "vrex_2014.csv",
    "vrex_2019.csv"
]

version = "original"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

# Load Components

In [ ]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/tcc_experiments_az_ml/src/components/data_prep.yaml")

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

In [ ]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/tcc_experiments_az_ml/src/components/train.yaml")

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

In [ ]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def prep_data_and_train_defaults_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    learning_rate_to_train,
    flag_remove_values_by_percentage,
    percentage_to_remove_column
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage,
        percentage_to_remove_column,
    )

    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=learning_rate_to_train,  # note: using a pipeline input as parameter
    )


In [ ]:
pipelines = []
percentages = [0.5, 0.6, 0.7, 0.8, 0.85, 0.90, 0.95, 0.97]

for percentage in percentages:
    for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):
        pipeline = prep_data_and_train_defaults_pipeline(
            data_to_train=Input(type="uri_file", path=data_to_train.path),
            data_to_test=Input(type="uri_file", path=data_to_test.path),
            flag_remove_null_values=True,
            learning_rate_to_train=0.05,
            flag_remove_values_by_percentage=True,
            percentage_to_remove_column=percentage,
        )
        
        pipelines.append(pipeline)

print("Quantidade de pipelines: ", len(pipelines))

In [ ]:
import datetime as dt

def _get_experiment_names() -> [str]:
    experiment_names = []
    for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
        current_time = dt.datetime.now()
        formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")  # Formata a data e hora atual
        train_name_base = train_name.split('.')[0]
        test_name_base = test_name.split('.')[0]
        name = f"{train_name_base}tested_{test_name_base}_executed_in_{formatted_time}_removed_null_values_by_percentage"
        experiment_names.append(name)
        print(name)
    return experiment_names


In [ ]:
experiment_names = _get_experiment_names()
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )

    ml_client.jobs.stream(pipeline_job.name)